In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import jieba
import numpy as np
from sklearn.impute import SimpleImputer

# 1 特征中含有字符串的时候（当成类别），如何做特征提取

In [6]:
def dictvec():
    """
    字典数据提取
    :return: None
    """
    # 实例化
    # sparse改为True，输出的是每个不为零位置的坐标，稀疏矩阵可以节省存储空间
    # 矩阵中存在大量的0，sparse存储只记录非零位置，节省空间的作用
    #Vectorizer中文含义是矢量器的含义
    dict1 = DictVectorizer(sparse=True)  # 把sparse设置为True，输出的是稀疏矩阵
    # 每个样本是一个字典，字典的key是特征名，value是特征值，有三个样本
    #调用fit_transform方法，把字典转换为稀疏矩阵
    data = dict1.fit_transform([{'city': '北京', 'temperature': 100},
                                {'city': '上海', 'temperature': 60},
                                {'city': '深圳', 'temperature': 30}])
    print(data)
    print('-' * 50)
    # 字典中一些类别数据，分别进行转换成特征
    print(dict1.get_feature_names_out())
    print("-" * 50)
    print(dict1.inverse_transform(data))  # 去看每个特征代表的含义，逆转回去

    return None


dictvec()

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (3, 4)>
  Coords	Values
  (0, 1)	1.0
  (0, 3)	100.0
  (1, 0)	1.0
  (1, 3)	60.0
  (2, 2)	1.0
  (2, 3)	30.0
--------------------------------------------------
['city=上海' 'city=北京' 'city=深圳' 'temperature']
--------------------------------------------------
[{'city=北京': np.float64(1.0), 'temperature': np.float64(100.0)}, {'city=上海': np.float64(1.0), 'temperature': np.float64(60.0)}, {'city=深圳': np.float64(1.0), 'temperature': np.float64(30.0)}]


# 一段英文文本如何变为数值类型

In [7]:
def couvec():
    """
    实例化CountVectorizer
    max_df,min_df整数，指每个词的所有文档中出现的最大、最小次数，超过这个次数的词会被过滤掉
    max_df,min_df小数(0-1之间的)：某个词的出现的次数占所有文档的比例，超过这个比例的词会被过滤掉
    min_df=2，过滤掉出现次数小于2的词
    默认会去除单个字母的单词，默认认为这个词对整个样本没有影响，认为其没有语义
    :return: 
    """
    vector = CountVectorizer(min_df=2)

    # 调用fit_transform方法，输入并转换数据,把文本转换为稀疏矩阵

    res = vector.fit_transform(
        ["life is short, i like python life",
        "life is too long, i dislike python",
        "life is short"])

    # 打印结果，把每个分词都分离了
    print(vector.get_feature_names_out())
    print('-'*50)
    print(res)
    print('-'*50)
    print(type(res))
    # 对照feature_names，标记每个词出现的次数
    print('-'*50)
    print(res.toarray()) #稀疏矩阵转换为数组
    print('-'*50)
    #拿每个样本里的特征进行显示
    print(vector.inverse_transform(res))

couvec()

['is' 'life' 'python' 'short']
--------------------------------------------------
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 10 stored elements and shape (3, 4)>
  Coords	Values
  (0, 1)	2
  (0, 0)	1
  (0, 3)	1
  (0, 2)	1
  (1, 1)	1
  (1, 0)	1
  (1, 2)	1
  (2, 1)	1
  (2, 0)	1
  (2, 3)	1
--------------------------------------------------
<class 'scipy.sparse._csr.csr_matrix'>
--------------------------------------------------
[[1 2 1 1]
 [1 1 1 0]
 [1 1 0 1]]
--------------------------------------------------
[array(['life', 'is', 'short', 'python'], dtype='<U6'), array(['life', 'is', 'python'], dtype='<U6'), array(['life', 'is', 'short'], dtype='<U6')]


# 一段汉字文本如何数值化，对于汉字不能用空格来分割

In [8]:
def countvec():
    """
    对文本进行特征值化，单个汉字单个字母不统计，因为单个汉字字母没有意义
    :return: None
    """
    cv = CountVectorizer()
    
    data = cv.fit_transform(["人生苦短，我 喜欢 python python", "人生漫长，不用 python"])
    
    print(cv.get_feature_names_out())
    print('-' * 50)
    print(data) # 稀疏存储，只记录非零位置
    print('-' * 50)
    print(data.toarray())
    
    return None

countvec()

['python' '不用' '人生漫长' '人生苦短' '喜欢']
--------------------------------------------------
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6 stored elements and shape (2, 5)>
  Coords	Values
  (0, 3)	1
  (0, 4)	1
  (0, 0)	2
  (1, 0)	1
  (1, 2)	1
  (1, 1)	1
--------------------------------------------------
[[2 0 0 1 1]
 [1 1 1 0 0]]
